# Projection matrix

In [28]:
import numpy as np


def project_3d_to_2d(X: np.ndarray, P: np.ndarray) -> np.ndarray:
    """Project 3D points to 2D."""
    assert X.shape[0] == 4
    assert P.shape == (3, 4)
    x = np.dot(P, X)
    return x / x[2]


K = np.array([[1000., 0., 500.],
              [0, 1000., 300.],
              [0, 0, 1.]])
# Rt = np.array([[1, 0, 0, 50],
#                [0, 1, 0, 40],
#                [0, 0, 1, 30]])
Rt = np.array([[np.cos(np.pi / 4), -np.sin(np.pi / 4), 0, 50],
               [np.sin(np.pi / 4), np.cos(np.pi / 4), 0, 40],
               [0, 0, 1, 30]])
P = K @ Rt

print(P)

x = project_3d_to_2d(np.array([0, 0, 0, 1]), P)
print(x)
# [2.16666667e+03 1.63333333e+03 1.00000000e+00]

[[ 7.07106781e+02 -7.07106781e+02  5.00000000e+02  6.50000000e+04]
 [ 7.07106781e+02  7.07106781e+02  3.00000000e+02  4.90000000e+04]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.00000000e+01]]
[2.16666667e+03 1.63333333e+03 1.00000000e+00]


In [31]:
import numpy as np

X1 = np.array([1, 1, 1, 1])
X2 = np.array([1, 1, 1.5, 1])

P @ X1

array([6.55000000e+04, 5.07142136e+04, 3.10000000e+01])

In [32]:
P @ X2

array([6.57500000e+04, 5.08642136e+04, 3.15000000e+01])

# Camera Center

In [27]:
import scipy.linalg as linalg
from typing import Tuple


def factor_camera_matrix(P: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Factorize the camera matrix into K, R, t."""
    K, R = linalg.rq(P[:, :3])

    # Make sure R has positive determinant
    T = np.diag(np.sign(np.diag(K)))
    K = np.dot(K, T)
    R = np.dot(T, R)

    t = np.dot(linalg.inv(K), P[:, 3])
    return K, R, t


def camera_center(P: np.ndarray) -> np.ndarray:
    """Compute the camera center from the projection matrix."""
    K, R, t = factor_camera_matrix(P)
    c = -np.dot(R.T, t)
    return c


K_, R_, t_ = factor_camera_matrix(P)

print(f"K = K_: {np.allclose(K_, K)}")
print(f"R = R_: {np.allclose(R_, Rt[:, :3])}")

K = K_: True
R = R_: True


# Fundamental matrix

In [ ]:
def compute_fundamental(x1: np.ndarray, x2: np.ndarray) -> np.ndarray:
    """Computes the fundamental matrix from corresponding 
	points using the normalized 8 point algorithm. 
	Implementation by Solem 2012."""
    assert x1.shape[1] == x2.shape[1]

    n = x1.shape[1]
    A = np.zeros((n, 9))
    for i in range(n):
        A[i] = [
            x1[0, i] * x2[0, i],
            x1[0, i] * x2[1, i],
            x1[0, i] * x2[2, i],
            x1[1, i] * x2[0, i],
            x1[1, i] * x2[1, i],
            x1[1, i] * x2[2, i],
            x1[2, i] * x2[0, i],
            x1[2, i] * x2[1, i],
            x1[2, i] * x2[2, i]
        ]
    # Get the singular vector with the smallest singular value
    U, S, V = linalg.svd(A)
    F = V[-1].reshape(3, 3)  # Reshape a vector to 3x3 matrix

    # Forcing the F to be rank 2.
    U, S, V = linalg.svd(F)
    S[2] = 0
    F = np.dot(U, np.dot(np.diag(S), V))
    return F

In [7]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

U, S, V = linalg.svd(A)

In [8]:
print(U)

[[-0.21483724  0.88723069  0.40824829]
 [-0.52058739  0.24964395 -0.81649658]
 [-0.82633754 -0.38794278  0.40824829]]
